In [42]:
import numpy as np
import pandas as pd
from scipy.stats import mvn, norm, gaussian_kde
from scipy.optimize import fsolve
# import seaborn as sns
from scipy import integrate
import plotly
import matplotlib.pyplot as plt
import os
import time

In [2]:
pd.__version__

'0.23.0'

In [161]:
def simulation_v2(time_para, delay, period):
    start = time.time()

    # Read Portfolio and correlation matrix
    path = r'C:\Users\Javier\Documents\MEGA\Universitattt\Master\Thesis\CDS_data\factor_model'
    directory = 'time_' + str(time_para) + '_delay_' + str(delay)
    file = 'period_' + str(period) + '_portfolio.csv'

    Portfolio = pd.read_csv(os.path.join(path,directory,file), sep=',')
    # Read the correlation matrix from .csv
    Omega = pd.read_csv(r'C:\Users\Javier\Documents\MEGA\Universitattt\Master\Thesis\CDS_data\factor_model\Ioannis\Omega.csv', index_col = 0,header = 0)

    m = len(Portfolio)    # number of counterparties in the portfolio
    N = 10**6             # number of simulations
    p = Omega.shape[0]    # number of systematic factors

    # Now we get the beta, gamma (PDGSD), PD, EAD and LGD
    Beta = Portfolio[[col for col in list(Portfolio) if col.startswith('beta')]].values
    gamma = Portfolio['gamma'].values
    PD = Portfolio['PD'].values
    EAD = Portfolio['EAD'].values
    LGD = Portfolio['LGD'].values

    df_port = Portfolio[['SOV_ID','PD','EAD','LGD']]

    # Analytical Expected Loss
    EL_an = np.sum(PD*EAD*LGD)

    # Calibrate default thresholds with PDs
    d = norm.ppf(PD)
    # perform a Cholesky factorisation to sample normal distributed
    # numbers with covariaton matrix Omega
    L = np.linalg.cholesky(Omega)

    np.random.seed(10)
    # generate independent normals
    Z = np.random.standard_normal((p, N))

    # convert independent unit normals to correlated
    F = np.dot(L, Z)

    # idiosyncratic loading s.t. the returns are standard normal
    id_load = np.diagonal(np.sqrt(1-np.dot(np.dot(Beta,Omega),Beta.T)))
    epsilon = np.random.standard_normal((N, m))
    # Put everything together to get the returns
    X = np.dot(Beta,F) + (id_load*epsilon).T
    X_df = pd.DataFrame(np.dot(Beta,F) + (id_load*epsilon).T)

    # Calculate UL with no contagion
    # construct default indicator
    I = (((X.T-d)<0))
    I_df = pd.DataFrame(I)
    L = (EAD*LGD*I).T
    
    # print(np.mean(L,axis=1))
    Loss=np.sum(L,axis=0)

    # Calculate UL with contagion

    SOV_ID = Portfolio['SOV_ID'].values
    SOV_LINK = Portfolio['SOV_LINK'].values

    df_d = pd.DataFrame(np.zeros((m,3)), columns = ['SOV_ID','Dsd','Dnsd'])
    df_d['SOV_ID']=SOV_ID

    PDs = df_port[df_port['SOV_ID']==1]['PD'].values[0]

    Dsd = np.zeros(m)
    Dnsd = np.zeros(m)

    # With contagion
    for i in range(0,m):
        if SOV_ID[i] != 0:
            Dsd[i] = d[i]
            Dnsd[i] = d[i]
        else:
            sov_ind = np.nonzero(SOV_ID == SOV_LINK[i])[0][0]
            PDs = PD[sov_ind]
            corr = np.dot(np.dot((Beta[i]).T,Omega),(Beta[sov_ind]))

            Fsd = lambda x: mvn.mvndst([-100, -100],\
                [x, norm.ppf(PDs)],[0,0],corr)[1] / PDs - gamma[i]
            Dsd[i] = fsolve(Fsd, norm.ppf(gamma[i])) # is there a better initial guess?
            Fnsd = lambda x: mvn.mvndst([-100, norm.ppf(PDs)],\
                [x, 100],[0,1],corr)[1] - PD[i] + gamma[i]*PDs
            Dnsd[i] = fsolve(Fnsd, norm.ppf(PD[i])) # is there a better initial guess?
            if Dsd[i]< d[i] or PD[i]<PD[sov_ind]:
                Dsd[i] = d[i]
                Dnsd[i] = d[i]

    df_d['Dsd'] = Dsd
    df_d['Dnsd'] = Dnsd

    # Thresholds
    D = np.array([Dnsd]*N).T
    D_df = pd.concat([df_d['Dnsd']]*N,axis = 1)
    D_df.columns = range(N)

    X2 = X_df.transpose()
    D2 = D_df.transpose()

    sov_ind = df_d[df_d['SOV_ID']==1].index[0]

    X_SD = X2[X2[sov_ind]<df_d.loc[sov_ind, 'Dsd']].copy()

    X_NSD = X2.drop(X_SD.index, axis = 0)


    I_SD = X_SD.lt(df_d['Dsd'], axis = 1)
    I_NSD = X_NSD.lt(df_d['Dnsd'],axis = 1)

    I_c = pd.concat([I_SD,I_NSD], axis = 0)

    I_aux = np.array(I_c)

    L = (EAD * LGD * I_aux)

    Loss_c = np.sum(L,axis=1)

    EL = np.mean(Loss)
    # Arithmetic mean of Loss
    EL_c = np.mean(Loss_c)

    # UL_98 = np.percentile(Loss, 98)
    UL_99 = np.percentile(Loss, 99)
    UL_995 = np.percentile(Loss, 99.5)
    UL_999 = np.percentile(Loss, 99.9)
    UL_9999 = np.percentile(Loss, 99.99)

    # UL_98_c = np.percentile(Loss_c, 98)
    UL_99_c = np.percentile(Loss_c, 99)
    UL_995_c = np.percentile(Loss_c, 99.5)
    UL_999_c = np.percentile(Loss_c, 99.9)
    UL_9999_c = np.percentile(Loss_c, 99.99)


    UL = np.array([ UL_99, UL_995, UL_999, UL_9999])
    UL_c = np.array([ UL_99_c, UL_995_c, UL_999_c, UL_9999_c])
    
    end = time.time()

    print(end-start)

    return UL, UL_c


In [ ]:
pd.options.mode.chained_assignment = None

In [171]:
path = r'C:\Users\Javier\Documents\MEGA\Universitattt\Master\Thesis\Results\Capital_periods'
folder1 = r'results_delay_'+str(delay)+'_per_'+str(per1)
folder2 = r'results_delay_'+str(delay)+'_per_'+str(per2)

no_con_1 = pd.read_csv(os.path.join(path,folder1,\
    'No_contagion_'+str(delay)+'_'+str(per1)+'.csv'),index_col = 0)
con_1 = pd.read_csv(os.path.join(path,folder1,\
    'Contagion_'+str(delay)+'_'+str(per1)+'.csv'),index_col = 0)
no_con_2 = pd.read_csv(os.path.join(path,folder2,\
    'No_contagion_'+str(delay)+'_'+str(per2)+'.csv'),index_col = 0)
con_2 = pd.read_csv(os.path.join(path,folder2,\
    'Contagion_'+str(delay)+'_'+str(per2)+'.csv'),index_col = 0)

In [189]:
to_perc = lambda x: "{0:.2f}".format(x)

In [190]:
comp = (con_2-con_1)/con_1*100

In [193]:
for c in comp.columns:
    for i in comp.index:
        comp.loc[i,c] = to_perc(comp.loc[i,c])

In [200]:
path = r'C:\Users\Javier\Documents\MEGA\Universitattt\Master\Thesis\Results\Probabilities'
files_1 = sorted([f for f in os.listdir(path) if \
		f.endswith('_delay_'+str(delay)+'_per_'+str(per1))])
files_2 = sorted([f for f in os.listdir(path) if \
		f.endswith('_delay_'+str(delay)+'_per_'+str(per2))])

In [201]:
files_1

['time_10_delay_1_per_0', 'time_15_delay_1_per_0', 'time_20_delay_1_per_0']

In [202]:
files_2

['time_10_delay_1_per_5', 'time_15_delay_1_per_5', 'time_20_delay_1_per_5']